In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install -q peft accelerate bitsandbytes datasets

In [ ]:
import torch
import gc

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)
from peft import PeftModel, PeftConfig, AutoPeftModelForCausalLM
from datasets import load_dataset

In [ ]:
# Download fine-tuned model
ft_model_name = "winirrr/typhoon-7b-quote-demoV01"

ft_model = AutoModelForCausalLM.from_pretrained(
    ft_model_name,
    return_dict=True,
    load_in_4bit=True,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.57G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

In [ ]:
# Download tokenizer from base model (typhoon-7b)
base_model_name = "scb10x/typhoon-7b"

tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                          padding_side="left")

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/563k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.99M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
print(ft_model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(35219, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplac

In [ ]:
def text_generator(model, prompt):
  inputs = tokenizer(prompt, return_tensors="pt")
  model.eval()
  with torch.inference_mode():
    outputs = model.generate(**inputs, penalty_alpha=0.6, top_k=4) #
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

In [ ]:
prompt = input("พิมโลดด:")

text_generator(model=ft_model,
              prompt= prompt)

พิมโลดด:โสก


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.

['โสกี้สีชมพูคือเรา ส่วนคนในใจเดียวนั้นคือเธอใช่ไหม?']


In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://hzzvzbwfknh-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from flask import Flask, render_template, request, send_from_directory
from transformers import pipeline, AutoModelForCausalLM

app = Flask(__name__,template_folder='/content', static_folder='/static')


# Define text generation pipeline
text_generator = pipeline("text-generation", model=ft_model, tokenizer=tokenizer)

#css
@app.route('/css/<path:path>')
def send_css(path):
    return send_from_directory('css', path)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/generate', methods=['POST'])
def generate():
    # Get input text from form
    prompt = request.form['input_text']

    # Generate text based on input
    generated_text = text_generator(prompt, max_length=50, do_sample=False)[0]['generated_text']

    return render_template('index.html', input_text=prompt, generated_text=generated_text)


app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Apr/2024 07:21:15] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2024 07:21:16] "GET /style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2024 07:21:16] "GET /static/styles/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2024 07:21:16] "GET /favicon.ico HTTP/1.1" 404 -
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [15/Apr/2024 07:21:34] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2024 07:21:35] "GET /style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2024 07:21:35] "GET /static/styles/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2024 07:21:35] "GET /favicon.ico HTTP/1.1" 404 -
Setting `pad_token_id` to `eos_token_id`: